In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Import data

In [2]:
df = pd.read_csv("./data/boston.csv")

Input features in order:
1) CRIM: per capita crime rate by town
2) ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
3) INDUS: proportion of non-retail business acres per town
4) CHAS: Charles River dummy variable (1 if tract bounds river; 0 otherwise)
5) NOX: nitric oxides concentration (parts per 10 million) [parts/10M]
6) RM: average number of rooms per dwelling
7) AGE: proportion of owner-occupied units built prior to 1940
8) DIS: weighted distances to five Boston employment centres
9) RAD: index of accessibility to radial highways
10) TAX: full-value property-tax rate per $10,000 [$/10k]
11) PTRATIO: pupil-teacher ratio by town
12) B: The result of the equation B=1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
13) LSTAT: % lower status of the population

Output variable:
1) MEDV: Median value of owner-occupied homes in $1000's [k$]



In [3]:
# get familiar with the data, check the shape, the first 5 rows, df.describe(), df.info()
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 516 entries, 0 to 533
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     516 non-null    float64
 1   ZN       516 non-null    float64
 2   INDUS    516 non-null    float64
 3   CHAS     516 non-null    int64  
 4   NOX      516 non-null    float64
 5   RM       516 non-null    float64
 6   AGE      516 non-null    float64
 7   DIS      516 non-null    float64
 8   RAD      516 non-null    float64
 9   TAX      516 non-null    float64
 10  PTRATIO  516 non-null    float64
 11  B        516 non-null    float64
 12  LSTAT    516 non-null    float64
 13  MEDV     516 non-null    float64
dtypes: float64(13), int64(1)
memory usage: 60.5 KB


# Data cleaning

## Remove NaN values

In [5]:
df = df.dropna()

df.describe()

In [11]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000
mean,3.649998,10.901381,11.104753,0.067061,0.554869,6.279884,68.965680,3.761168,9.625247,407.783037,18.493294,358.197751,12.754004,22.379487
std,8.550025,22.643310,6.787953,0.250375,0.115699,0.711077,28.024461,2.083707,8.740754,169.131823,2.141302,88.521328,7.159643,9.149760
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082865,0.000000,5.190000,0.000000,0.449000,5.887500,45.050000,2.082850,4.000000,277.000000,17.400000,375.695000,7.160000,16.700000
50%,0.253560,0.000000,9.690000,0.000000,0.538000,6.223000,77.000000,3.215700,5.000000,330.000000,19.100000,391.700000,11.480000,21.200000
75%,3.735965,12.500000,18.100000,0.000000,0.631000,6.617000,94.300000,5.108550,24.000000,666.000000,20.200000,396.240000,17.115000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


# Handle outliers

In [10]:
# Did you notice anything weird in df.describe()? If not, check out MEDV's max value, the min value and the mean value. 
# What's going on? Is this a problem? How can you fix it?
df = df[df["MEDV"] > 0]
df = df[df["MEDV"] < 30000]

## Remove duplicates

In [12]:
# HINT: there is a function in pandas made just for this purpose
# HINT 2: https://google.gprivate.com/search.php?search?q=pandas+remove+duplicates
df = df.drop_duplicates()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 533
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     480 non-null    float64
 1   ZN       480 non-null    float64
 2   INDUS    480 non-null    float64
 3   CHAS     480 non-null    int64  
 4   NOX      480 non-null    float64
 5   RM       480 non-null    float64
 6   AGE      480 non-null    float64
 7   DIS      480 non-null    float64
 8   RAD      480 non-null    float64
 9   TAX      480 non-null    float64
 10  PTRATIO  480 non-null    float64
 11  B        480 non-null    float64
 12  LSTAT    480 non-null    float64
 13  MEDV     480 non-null    float64
dtypes: float64(13), int64(1)
memory usage: 56.2 KB


# Model creation

In [14]:
# Create the XGBoost regression model. XGBoost stands for: eXtreme Gradient Boosting. 
# This is a very popular algorithm, used in machine learning competitions and in the industry. 
# We will use it for regression, but it can also be used for classification.

model = xgb.XGBRegressor()

In [19]:
# let y be the target column, and X be the rest of the df
X = df.drop("MEDV", axis=1)
y = df["MEDV"]

In [21]:
# Split the data into train and test sets with the function train_test_split from sklearn. Use test_size=0.2 and random_state=42
# We use train_test_split to split the data into train and test sets. We will use the train set to train the model, and the test set to evaluate the model.
# The reason we need a test set is to be able to evaluate the model. If we train the model on the whole dataset, 
# it will learn the dataset perfectly, but we will not know how it performs on unseen data. 

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [22]:
# use the training set (X_train, y_train) to train the model by calling the .fit() method
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [23]:
# Use the model to predict the target values for the test set (X_test)
preds = model.predict(X_test)

In [25]:
# find the mean squared error for the predictions (a value to see the value of the predictions, lower is better)
# find the error between the y_test and the preds
mse = mean_squared_error(preds, y_test)

In [27]:
# print the mse to see how much, on average, your model is off (squared)
mse

10.68097416865667

# Hyperparameter tuning

In [31]:
# These are some of the hyperparameters you can tune for XGBoost. 
# A hyperparameter is a parameter that is not learned by the model, but is set by the user.
# The parameters that are learned by the model are called model parameters.
# The model starts off with some default values for the hyperparameters, but you can change them to get potentially better results.
# This process is called hyperparameter tuning.

# If you want, you can adjust the hyperparameters and see if you can get a better result. You can also add more hyperparameters to the dictionary.
# List of hyperparameters: https://xgboost.readthedocs.io/en/latest/parameter.html
params = {
    "learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    "max_depth": [3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight": [1, 3, 5, 7],
    "gamma": [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample_bytree": [0.3, 0.4, 0.5, 0.7],
    "n_estimators": [100, 200, 300, 400, 500, 900, 1100, 1500],
}

In [41]:
# Use RandomizedSearchCV to find the best hyperparameters for the model. There are other ways to do this, but random search will work for this purpose.
# Random search is a method for hyperparameter tuning that will try a given number of random combinations of hyperparameters.
# Use the training set (X_train, y_train) to instantiate the random search by calling the .fit() method with the test set
# HINT: n_iter is the number of iterations to run the random search, if this number is too high, it will take a long time to run, 
# but if it's too low, it will not find the best hyperparameters. You should try to find a happy medium.

# First, create a new, similar model, but with the default hyperparameters. Do not fit this model with the training set.
model2 = xgb.XGBRegressor()

random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=250, scoring="neg_mean_squared_error", n_jobs=-1, cv=5)

# Fit the model with x and y train sets
random_search.fit(X_train, y_train)

/opt/miniconda3/envs/dave3625/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          callbacks=None, colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, gamma=0, gpu_id=-1,
                                          grow_policy='depthwise',
                                          importance_type=None,
                                          interaction_constraints='',
                                          learning_rate=0.300000012,
                                          max_bin=256, m...
                                          num_parallel_tree=1, predictor='auto',
                                          random_state=0, reg_alpha=0,
                                          reg_lambda=1, ...),
                   n_iter=250, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5,
                                                             0.7],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2,
                                                          0.25, 0.3],
                                        'max_depth': [3, 4, 5, 6, 8, 10, 12,
                                                      15],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 900, 1100,
                                                         1500]},
                   scoring='neg_mean_squared_error')

In [42]:
# Retrieve the best model/estimator from the random search
model_new = random_search.best_params_
model_new

{'n_estimators': 300,
 'min_child_weight': 1,
 'max_depth': 10,
 'learning_rate': 0.05,
 'gamma': 0.0,
 'colsample_bytree': 0.7}

In [43]:
model_new = random_search.best_estimator_

In [44]:
type(model_new)

xgboost.sklearn.XGBRegressor

In [45]:
# Create new predictions with the new model
preds = model_new.predict(X_test)

In [46]:
# Get the new mean square error
mse_new = mean_squared_error(preds, y_test)

mse_new

10.33695472617273

In [47]:
print(f"relation between better error on the new model and the old error: {(mse_new / mse)}")

# If the new model did not perform better, this means that the default hyperparameters were better, but it is highly likely that even better ones exist.
# You can try to run the random search again, but with more iterations, or you can try to use GridSearchCV instead of RandomizedSearchCV ot test _every_ combination of hyperparameters.
# You can also edit the hyperparameters in the dictionary to see if you can get better results.

relation between better error on the new model and the old error: 0.9677913796015474
